# Projeto 2 - Ciência dos Dados

Nome: Gabriela Choichit Giosa

Nome: João Pedro Faria de Araujo

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [8]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import pandas as pd
import numpy as np
import re
from IPython.display import display
import string

In [2]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

+___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [11]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = '#Joker'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

lendo o arquivo e fazendo a tabela de relativos e não relativos

In [27]:
joker = pd.read_excel('#JokerFull.xlsx')
teste = pd.read_excel('#JokerFull.xlsx', 'Teste')
joker_idx = joker.set_index('Treinamento')
joker_relevante = joker_idx[joker_idx.Classe == 1]
joker_nao_relevante = joker_idx[joker_idx.Classe == 0]
teste_idx = teste.set_index('Teste')
teste_idx_2 = set(teste_idx.index)

filtrando o teste

In [28]:
teste_palavras = str(teste_idx_2)
teste_palavras = cleanup(teste_palavras)
teste_palavras = teste_palavras.split()

joker total

In [9]:
lista_palavras = str(set(joker_idx.index))
lista_palavras =cleanup(lista_palavras)
lista_palavras = lista_palavras.split()
lista_palavras

["{'stealthy",
 'android',
 'trojan',
 'spy',
 'signs',
 'you',
 'up',
 'for',
 'premium',
 'subscriptions\\nvia',
 '@bleepincomputer\\n#proficio',
 '#threatnews',
 '#cybersecurity',
 '#mssp',
 '#mdr',
 '#joker',
 '#trojan\\nhttps',
 '//t',
 "co/muyi7huwgn'",
 "'rt",
 '@mysteriostephen',
 'the',
 'dark',
 'knight',
 'promotional',
 '#photoshoot',
 '"',
 '\\n#heathledger',
 '#joker',
 'a',
 'true',
 'legend',
 '"',
 '🃏📸\\n@spider_symbiote',
 'https',
 '//t',
 "co/0kkd83…'",
 "'the",
 '@untitled_cast',
 '#tiff19\\xa0',
 'coverage',
 'continues',
 'with',
 'our',
 'review',
 'of',
 '#joker',
 '\\n\\n40',
 'minutes',
 'of',
 'spoiler',
 'free',
 'impressions',
 '…',
 'https',
 '//t',
 "co/f6fsxddc9z'",
 '"rt',
 '@comicbook',
 '#joker',
 'director',
 'explains',
 'why',
 "it's",
 'not',
 'a',
 'typical',
 'origin',
 'movie\\nhttps',
 '//t',
 'co/qqh3phmwzh',
 'https',
 '//t',
 'co/sfedovmgvx"',
 '"chinese',
 'citizens',
 'find',
 'trump',
 "'perplexing'",
 'on',
 'trade',
 'war',
 'china',


In [10]:
serie_total = pd.Series(lista_palavras)
tabela_total_absoluta = serie_total.value_counts()
tabela_total_relativa = serie_total.value_counts(True)
tabela_total_relativa

//t                            0.035023
#joker                         0.032944
https                          0.031185
the                            0.028946
'rt                            0.021430
a                              0.017592
to                             0.014393
of                             0.011674
is                             0.011035
and                            0.008316
for                            0.007836
i                              0.007676
this                           0.007197
in                             0.006877
you                            0.006077
joker                          0.006077
on                             0.005757
movie                          0.005437
it                             0.005277
"rt                            0.004958
#jokermovie                    0.004958
joaquin                        0.003838
be                             0.003838
with                           0.003838
that                           0.003838


In [11]:
lista_palavras_relevantes = str(set(joker_relevante.index))
lista_palavras_relevantes = cleanup(lista_palavras_relevantes)
lista_palavras_relevantes = lista_palavras_relevantes.split()
lista_palavras_relevantes

["{'rt",
 '@a__g__t',
 'instant',
 'classic',
 '#joker',
 '#jock',
 'https',
 '//t',
 "co/zysqqsq0dk'",
 "'#joker",
 'seems',
 'like',
 'the',
 'go',
 'to',
 'movie',
 'for',
 'all',
 'future',
 'mass',
 'shooters',
 "'",
 "'@comicbook",
 'injustice',
 'fall',
 "#joker'",
 "'hit",
 'the',
 'feed',
 'for',
 'the',
 'latest',
 'tnr',
 'episode',
 'where',
 'we',
 'are',
 'talking',
 '#joker',
 '#birdsofprey',
 '#themandalorian',
 '#starwars',
 'triple',
 'force',
 'friday',
 'and',
 'much',
 'more',
 '#wethenerd',
 '#staynerd',
 '#weareallclowns',
 '\\n\\n🎧',
 'on',
 '@podbeancom',
 '@applepodcasts',
 '@spotify',
 'or',
 'hit',
 'the',
 'link',
 'below',
 '\\n\\nhttps',
 '//t',
 'co/vnhd1b5i0b',
 'https',
 '//t',
 "co/lxznxttqop'",
 "'rt",
 '@cbr',
 '#jokermovie',
 'makes',
 'a',
 'questionable',
 'soundtrack',
 'selection',
 'https',
 '//t',
 'co/zxpftzdd1y',
 '#joker',
 'https',
 '//t',
 "co/tlzjugaeps'",
 "'#joker",
 '❤️',
 'waiting',
 '😁😁😁😁',
 'https',
 '//t',
 "co/3mn17gfhc8'",
 "'r

In [12]:
lista_palavras_relevantes2 = str(set(joker_relevante.index))

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
lista_palavras_relevantes2 = lista_palavras_relevantes2.translate(translator).split()
lista_palavras_relevantes2

#for k in range(len(lista_palavras_relevantes)):
      #if lista_palavras_relevantes[k] not in lista_palavras_relevantes2:
        #print(lista_palavras_relevantes[k])
lista_palavras_relevantes2

['rt',
 'a',
 'g',
 't',
 'instant',
 'classic',
 'joker',
 'jock',
 'https',
 't',
 'co',
 'zysqqsq0dk',
 'joker',
 'seems',
 'like',
 'the',
 'go',
 'to',
 'movie',
 'for',
 'all',
 'future',
 'mass',
 'shooters',
 'comicbook',
 'injustice',
 'fall',
 'joker',
 'hit',
 'the',
 'feed',
 'for',
 'the',
 'latest',
 'tnr',
 'episode',
 'where',
 'we',
 'are',
 'talking',
 'joker',
 'birdsofprey',
 'themandalorian',
 'starwars',
 'triple',
 'force',
 'friday',
 'and',
 'much',
 'more',
 'wethenerd',
 'staynerd',
 'weareallclowns',
 'n',
 'n🎧',
 'on',
 'podbeancom',
 'applepodcasts',
 'spotify',
 'or',
 'hit',
 'the',
 'link',
 'below',
 'n',
 'nhttps',
 't',
 'co',
 'vnhd1b5i0b',
 'https',
 't',
 'co',
 'lxznxttqop',
 'rt',
 'cbr',
 'jokermovie',
 'makes',
 'a',
 'questionable',
 'soundtrack',
 'selection',
 'https',
 't',
 'co',
 'zxpftzdd1y',
 'joker',
 'https',
 't',
 'co',
 'tlzjugaeps',
 'joker',
 '❤️',
 'waiting',
 '😁😁😁😁',
 'https',
 't',
 'co',
 '3mn17gfhc8',
 'rt',
 'ashleygmenzel

In [13]:

#lista_1 = list(set(joker_relevante.index))
#z = len(list(set(joker_relevante.index)))
#lista_palavras_relevantes2 = []

#for i in range(z):
#    lista_palavras_relevantes2 = lista_palavras_relevantes2 + lista_1[i].split()
#print(len(lista_palavras_relevantes2))
#lista_palavras_relevantes2 = [str(i) for i in lista_palavras_relevantes2]
#lista_palavras_relevantes2 = [''.join(c for c in s if c not in string.punctuation) for s in lista_palavras_relevantes2]
#lista_palavras_relevantes2 = [s for s in lista_palavras_relevantes2 if s]
 


#for k in range(len(lista_palavras_relevantes)):
#      if lista_palavras_relevantes[k] not in lista_palavras_relevantes2:
#        print(lista_palavras_relevantes[k])

Frequencias relevante relativas e absolutas

In [14]:
serie_relevante = pd.Series(lista_palavras_relevantes + lista_palavras + teste_palavras)
tabela_relevante_absoluta = serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa

//t                                  0.034096
#joker                               0.033394
https                                0.030869
the                                  0.028764
'rt                                  0.022310
a                                    0.017539
to                                   0.013610
of                                   0.012207
is                                   0.011997
for                                  0.008910
and                                  0.008629
i                                    0.008489
this                                 0.007437
in                                   0.007296
movie                                0.005963
joker                                0.005612
on                                   0.005542
#jokermovie                          0.005402
joaquin                              0.004700
it                                   0.004700
you                                  0.004490
as                                

lista com as palavras não relevantes filtradas

In [17]:
lista_palavras_nao_relevantes = str(set(joker_nao_relevante.index))
lista_palavras_nao_relevantes =cleanup(lista_palavras_nao_relevantes)
lista_palavras_nao_relevantes = lista_palavras_nao_relevantes.split()
lista_palavras_nao_relevantes

["{'rt",
 '@mysteriostephen',
 'the',
 'dark',
 'knight',
 'promotional',
 '#photoshoot',
 '"',
 '\\n#heathledger',
 '#joker',
 'a',
 'true',
 'legend',
 '"',
 '🃏📸\\n@spider_symbiote',
 'https',
 '//t',
 "co/0kkd83…'",
 "'stealthy",
 'android',
 'trojan',
 'spy',
 'signs',
 'you',
 'up',
 'for',
 'premium',
 'subscriptions\\nvia',
 '@bleepincomputer\\n#proficio',
 '#threatnews',
 '#cybersecurity',
 '#mssp',
 '#mdr',
 '#joker',
 '#trojan\\nhttps',
 '//t',
 "co/muyi7huwgn'",
 "'rt",
 '@yousmmashogun',
 '@gamebredfighter',
 'give',
 'them',
 '#joker',
 '#gamebred',
 '🔥🔥',
 'https',
 '//t',
 "co/iaizpksd8z'",
 "'rt",
 '@afis8',
 'gonna',
 'have',
 'to',
 'sit',
 'with',
 '#joker',
 'for',
 'a',
 'little',
 'while',
 'but',
 'i',
 'will',
 'say',
 'this',
 'a',
 'lot',
 'of',
 'incels',
 'are',
 'going',
 'to',
 'feel',
 'pretty',
 'fucking',
 "empower…'",
 '\'"slow',
 'burn"\\ntoddphillips1\\n\\none',
 'month',
 'till\\xa0#joker',
 'uk',
 'poster',
 '@',
 'auckland',
 'cbd',
 'https',
 '//

Frequencias nao relevantes relativas e absolutas

In [18]:
serie_nao_relevante = pd.Series(lista_palavras_nao_relevantes + lista_palavras + teste_palavras)
tabela_nao_relevante_absoluta = serie_nao_relevante.value_counts()
tabela_nao_relevante_relativa = serie_nao_relevante.value_counts(True)
tabela_nao_relevante_relativa

//t               0.033498
#joker            0.031827
https             0.030460
the               0.028029
'rt               0.021269
a                 0.016331
to                0.013749
of                0.012457
is                0.011014
for               0.009267
and               0.008583
i                 0.007520
in                0.007216
this              0.006912
on                0.006532
joker             0.005469
you               0.005317
#jokermovie       0.004633
"rt               0.004558
movie             0.004330
it                0.004026
one               0.004026
joaquin           0.003798
be                0.003798
with              0.003722
#batman           0.003646
as                0.003342
that              0.003342
\n                0.003114
are               0.003038
                    ...   
xxl               0.000076
"what\'s          0.000076
homage            0.000076
#hotdogwater      0.000076
kind              0.000076
wildcard          0.000076
e

Frequencias nao relevantes e relevantes relativas e absolutas

Priors 
### P(n) e P(r)

In [19]:
Pn = tabela_nao_relevante_absoluta.sum()/tabela_total_absoluta.sum()
Pr = tabela_relevante_absoluta.sum()/tabela_total_absoluta.sum()
Pr, Pn

(2.2795458180073567, 2.1053894130817206)

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [20]:
len(joker_relevante.index[0].split())

12

In [21]:
len(joker_relevante.index)


262

In [29]:
tweet_relevante = []
tweet_nao_relevante = []


P_tweet_dado_relevante = []
P_tweet_dado_nao_relevante = []

for k in range(len(joker_relevante.index)):
    tweet_relevante.append(joker_relevante.index[k])    
for o in range(len(joker_nao_relevante.index)):
    tweet_nao_relevante.append(joker_nao_relevante.index[o])


for g in teste_idx.index:
    #print(g.split())
    P_tweet_dado_relevante.append(tabela_relevante_relativa[g.split()].prod()) 
    P_tweet_dado_nao_relevante.append(tabela_nao_relevante_relativa[g.split()].prod())
    print(g)



rt @mjfit: the #joker film is getting rave reviews for #joaquinphoenix’s performance. check out the latest trailer for the movie—plus every…
@att @atthelp @wbpictures @wbhomeent @warnermediagrp @warnerbrosuk @warnerbros

so the upcoming #jokermovie is coming which i think looks great.

i just wanted to ask when do you plan to #releasetheayercut? i would like to see more of this joker, please.

#suicidesquad2016
#joker https://t.co/vgxeqepaf7
rt @comicbook: #joker director explains why it's not a typical origin movie
https://t.co/qqh3phmwzh https://t.co/sfedovmgvx
can't wait for the new #joker movie
#joker did uh, winter soldier incite terrorism? did civil war encourage the public to riot on the streets and go against the government? did infinity war encourage someone to try to solve world hunger by eliminating 50% of earth's population? no? then move along. https://t.co/kvoziivvsh
rt @imsmokes: "professional" journalism in 2019 everyone. #joker
clowns writing reviews for clown movie. i

rt @gracerandolph: so #birdsofprey teaser has been confirmed as a theater exclusive with #itchapter2, but i'm hearing a full official tease…
rt @shallow_ss: joker (2019)
reviews from some of the biggest hollywood traits: 

ign: 10/10
variety: 5/5
hollywood reporter (thr): 4.5/5
e…
rt @rameshdesilva2: *whispers*lead the way! #joker #heathledger #joaquinphoenix #jaredleto #jokermovie #releasethesnydercut https://t.co/sr…
rt @starklokifan97: i cannot stress this enough of how much i love this performance, the joker will always be an iconic role, heath was an…
rt @thenerdrm: hit the feed for the latest tnr episode where we are talking; #joker, #birdsofprey, #themandalorian, #starwars triple force…
rt @comicbookstalk: 1 month #joker https://t.co/8isuoar9bn
rt @shallow_ss: joker (2019)
reviews from some of the biggest hollywood traits: 

ign: 10/10
variety: 5/5
hollywood reporter (thr): 4.5/5
e…
rt @berghain38: joker (2019)reviews:

ign: 10/10
variety: 5/5
hollywood reporter (thr): 4.5/5
emp

rt @variety: #joker is a "hypnotically perverse, ghoulishly grippingly urban-nightmare comic fantasia." read the full review: https://t.co/…
look...i've been torn on #joker since the first trailer...but the last 2 have really shown me what i wanted..&amp; i am incredibly excited for this film... it is for the darker reason's...so many scene's showing him being walked on, disrespected &amp; overall losing his shit.
rt @shallow_ss: joker (2019)
reviews from some of the biggest hollywood traits: 

ign: 10/10
variety: 5/5
hollywood reporter (thr): 4.5/5
e…
rt @berghain38: joker (2019)reviews:

ign: 10/10
variety: 5/5
hollywood reporter (thr): 4.5/5
empire magazine: 10/10
total film magazine: 1…
upcoming dc flicks to check off your watch list - https://t.co/ih0mfukavg #dcfilms #dc #dccomics #wonderwoman1984 #birdsofpray #joker #batman #suicide squad #aquaman #shazam #blackadam #theflash #galgadot #joaquinpheonix #robertpattinson #jasonmomoa #zacharylevi https://t.co/0rkyt9oidw
rt @thenerdrm

In [30]:
len(P_tweet_dado_relevante)

267

In [65]:
i = 0
j = 0
testando = []
teste_2 = []

for k in range(len(P_tweet_dado_relevante)):
    if P_tweet_dado_relevante[k] > P_tweet_dado_nao_relevante[k]:
        i = i + 1
        testando.append(1)
        if testando[k] == teste_idx.Classe[k]:
            teste_2.append('VP')
        else:
            teste_2.append('FN')
    if P_tweet_dado_relevante[k] == P_tweet_dado_nao_relevante[k]:
        print('q')
    elif P_tweet_dado_relevante[k] < P_tweet_dado_nao_relevante[k]:
        j = j + 1
        testando.append(0)
        if testando[k] == teste_idx.Classe[k]:
            teste_2.append('VN')
        else:
            teste_2.append('FP')
    
print(i, j)
#teste_2.value_counts()
print('Verdadeiro Positivo:',teste_2.count('VP'),'\nFalso Negativo:', teste_2.count('FN'),'\nVerdadeiro Negativo:', teste_2.count('VN'),'\nFalso Positivo:', teste_2.count('FP'))
for k in testando:
    print(k)

169 98
Verdadeiro Positivo: 143 
Falso Negativo: 26 
Verdadeiro Negativo: 53 
Falso Positivo: 45
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
0
0
1
0
1
1
1
1
1
0
1
0
1
1
1
1
0
0
0
0
0
0
0
0
1
0
1
0
1
1
1
1
1
0
0
0
1
0
0
0
0
1
0
1
1
1
1
0
1
1
1
1
0
0
1
1
1
0
0
0
1
0
0
1
1
1
1
0
1
1
1
0
0
1
0
1
0
1
1
1
1
1
0
1
1
0
0
0
1
0
1
1
1
0
1
1
1
1
1
1
0
0
1
0
0
0
0
1
0
1
0
1
1
1
1
1
1
0
0
0
1
1
0
1
1
1
0
0
1
1
0
1
0
1
0
0
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
0
0
1
1
0
1
1
0
1
0
0
0
1
0
0
0
0
1
0
0
1
0
1
1
0
1
1
1
0
1
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
0
0
0
1
1
0
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
0
1
1
0
1
0


In [78]:
taxa_acerto = (teste_2.count('VP') + teste_2.count('VN'))/(teste_2.count('VP') + teste_2.count('FP') + teste_2.count('VN') + teste_2.count('FN'))

print('taxa de acerto:', round(taxa_acerto, 2)*100)

taxa de acerto: 73.0


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**